In [1]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import (
    StreamingStdOutCallbackHandler
)
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [2]:
from langchain.vectorstores import Chroma

# Load Embeddings 

In [5]:
# embeddings = HuggingFaceInstructEmbeddings(
#         model_name="intfloat/multilingual-e5-large",
#         model_kwargs={"device": DEVICE},
#     )
# embedding engine
embeddings = HuggingFaceInstructEmbeddings()
# embeddings = HuggingFaceInstructEmbeddings(
#         model_name="hkunlp/instructor-xl",
#         model_kwargs={"device": DEVICE},
#     )

C:\Users\Administrator\anaconda3\envs\autogptq\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


# Initialize the Template

In [6]:
template = """Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Loading Model 

In [7]:
llm = LlamaCpp(
    # model_path="llama-2-7b-chat.ggmlv3.q2_K.bin",
    model_path="C:/Users/Alaa AI/Python Projects/Chatbots/GPT by me/llama-2-7b-chat.Q4_K_M.gguf",
    n_ctx=40000,
    n_gpu_layers=512,
    n_batch=30,
    callback_manager=callback_manager,
    temperature = 0.9,
    max_tokens = 30000,
    n_parts=1,
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [8]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Load Existing Knowledge Base 

In [35]:
updated_db_knowledge_base = Chroma(persist_directory="input", embedding_function=embeddings)

# Create New Knowledge base

In [15]:
knowledge_base_loader = PyPDFDirectoryLoader("pdfs")
knowledge_base = knowledge_base_loader.load()

In [16]:
knowledge_base_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
knowledge_base_texts = knowledge_base_text_splitter.split_documents(knowledge_base)
len(knowledge_base_texts)

243

In [17]:
%%time
updated_db_knowledge_base = Chroma.from_documents(knowledge_base_texts, embeddings, persist_directory="db-knowledge-base")

CPU times: total: 14 s
Wall time: 13 s


# Prepare Template using the context

In [36]:
query = "what is the date of birth of the examinee 'o'"
search = updated_db_knowledge_base.similarity_search(query, k=2)

In [37]:
template = '''Context: {context}

Based on Context provide me answer for following question
Question: {question}

Tell me the information about the fact. The answer should be from context only
do not use general knowledge to answer the query'''

prompt = PromptTemplate(input_variables=["context", "question"], template= template)
final_prompt = prompt.format(question=query, context=search)

In [38]:
llm_chain.run(final_prompt)

 Based

Llama.generate: prefix-match hit


 on the provided context, we can determine that the examinee's name is "o". However, there is no information in the given context regarding the date of birth of the examinee "o". Therefore, the answer to the question is "not found" or "unavailable".

' Based on the provided context, we can determine that the examinee\'s name is "o". However, there is no information in the given context regarding the date of birth of the examinee "o". Therefore, the answer to the question is "not found" or "unavailable".'

# Using Docx and Chroma

Adding File 1 to Chroma

In [31]:
loader = UnstructuredFileLoader("C:/Users/Alaa AI/Python Projects/Diagnosis/docx_input/1.docx")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
Chroma.from_documents(knowledge_base_texts, embeddings, persist_directory="input")

Adding File 2 to Chroma

In [32]:
loader = UnstructuredFileLoader("C:/Users/Alaa AI/Python Projects/Diagnosis/docx_input/2.docx")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
Chroma.from_documents(knowledge_base_texts, embeddings, persist_directory="input")

Adding File 3 to Chroma

In [33]:
loader = UnstructuredFileLoader("C:/Users/Alaa AI/Python Projects/Diagnosis/docx_input/3.docx")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
Chroma.from_documents(knowledge_base_texts, embeddings, persist_directory="input")

Adding File 4 to Chroma

In [34]:
loader = UnstructuredFileLoader("C:/Users/Alaa AI/Python Projects/Diagnosis/docx_input/4.docx")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
Chroma.from_documents(knowledge_base_texts, embeddings, persist_directory="input")

# Using Docx

In [9]:
loader = UnstructuredFileLoader("C:/Users/Alaa AI/Python Projects/Diagnosis/docx_input/1.docx")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [10]:
# embedding engine
hf_embedding = HuggingFaceInstructEmbeddings()

load INSTRUCTOR_Transformer
max_seq_length  512


In [11]:
db = FAISS.from_documents(docs, hf_embedding)

# save embeddings in local directory
db.save_local("faiss_AiArticle1")

# load from local
db = FAISS.load_local("faiss_AiArticle1/", embeddings=hf_embedding)

In [12]:
query = "what is the date of birth of the examinee 'o'"
search = db.similarity_search(query, k=2)

In [13]:
template = '''Context: {context}

Based on Context provide me answer for following question
Question: {question}

Tell me the information about the fact. The answer should be from context only
do not use general knowledge to answer the query'''

prompt = PromptTemplate(input_variables=["context", "question"], template= template)
final_prompt = prompt.format(question=query, context=search)

In [14]:
llm_chain.run(final_prompt)

 According to the provided document, the date of birth of the examinee 'o' is February 2nd, 2016.

" According to the provided document, the date of birth of the examinee 'o' is February 2nd, 2016."